In [ ]:
@file:DependsOn("ai.timefold.solver:timefold-solver-core:1.11.0")
@file:DependsOn("com.fasterxml.jackson.module:jackson-module-kotlin:2.17.1")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-datetime:0.5.0")

In [ ]:
import com.fasterxml.jackson.annotation.JsonFormat
import kotlin.math.sqrt

@JsonFormat(shape = JsonFormat.Shape.ARRAY)
data class Location(
    val latitude: Double,
    val longitude: Double
) {
    fun calcEuclideanDistanceTo(other: Location): Double {
        val latDiff = latitude - other.latitude
        val lonDiff = longitude - other.longitude
        return sqrt(latDiff * latDiff + lonDiff * lonDiff)
    }
}

In [ ]:
import kotlinx.datetime.LocalDateTime
import kotlinx.datetime.Duration

data class Visit(
    val id: String,
    val name: String,
    val location: Location,
    val demand: Int,
    val minStartTime: LocalDateTime,
    val maxEndTime: LocalDateTime,
    val serviceDuration: Duration
) {
    override fun toString(): String = name
}

In [ ]:
import ai.timefold.solver.core.api.domain.entity.PlanningEntity
import ai.timefold.solver.core.api.domain.variable.PlanningListVariable
import kotlinx.datetime.LocalDateTime

@PlanningEntity
data class Vehicle(
    val id: String,
    val name: String,
    val homeLocation: Location,
    val capacity: Int,
    val startTime: LocalDateTime
) {
    @PlanningListVariable
    var visits: MutableList<Visit> = mutableListOf()

    // No-arg constructor required for Timefold
    constructor() : this("", "", Location(0.0, 0.0), 0, LocalDateTime(1970, 1, 1, 0, 0))

    override fun toString(): String = name

    fun getTotalDemand(): Int = visits.sumOf { it.demand }

    fun getTotalTravelTime(): Duration {
        var totalTime = Duration.ZERO
        var currentLocation = homeLocation
        for (visit in visits) {
            totalTime += calculateTravelTime(currentLocation, visit.location)
            totalTime += visit.serviceDuration
            currentLocation = visit.location
        }
        totalTime += calculateTravelTime(currentLocation, homeLocation)
        return totalTime
    }

    private fun calculateTravelTime(from: Location, to: Location): Duration {
        val distance = from.calcEuclideanDistanceTo(to)
        // Assuming an average speed of 50 km/h
        return Duration.seconds((distance * 3600 / 50).toLong())
    }
}

In [ ]:
import ai.timefold.solver.core.api.domain.solution.PlanningEntityCollectionProperty
import ai.timefold.solver.core.api.domain.solution.PlanningScore
import ai.timefold.solver.core.api.domain.solution.PlanningSolution
import ai.timefold.solver.core.api.domain.solution.ProblemFactCollectionProperty
import ai.timefold.solver.core.api.domain.valuerange.ValueRangeProvider
import ai.timefold.solver.core.api.score.buildin.hardsoftlong.HardSoftLongScore
import kotlinx.datetime.LocalDateTime

@PlanningSolution
data class VehicleRoutePlan(
    val name: String,
    val startDateTime: LocalDateTime,
    val endDateTime: LocalDateTime,
    @PlanningEntityCollectionProperty
    val vehicles: List<Vehicle>,
    @ProblemFactCollectionProperty
    @ValueRangeProvider
    val visits: List<Visit>
) {
    @PlanningScore
    var score: HardSoftLongScore? = null

    // No-arg constructor required for Timefold
    constructor() : this("", LocalDateTime(1970, 1, 1, 0, 0), LocalDateTime(1970, 1, 1, 0, 0), emptyList(), emptyList())
}

In [ ]:
import ai.timefold.solver.core.api.score.buildin.hardsoftlong.HardSoftLongScore
import ai.timefold.solver.core.api.score.stream.Constraint
import ai.timefold.solver.core.api.score.stream.ConstraintFactory
import ai.timefold.solver.core.api.score.stream.ConstraintProvider
import kotlinx.datetime.Duration

class VehicleRoutingConstraintProvider : ConstraintProvider {
    override fun defineConstraints(constraintFactory: ConstraintFactory): Array<Constraint> {
        return arrayOf(
            vehicleCapacity(constraintFactory),
            timeWindowConstraint(constraintFactory),
            minimizeTravelTime(constraintFactory)
        )
    }

    fun vehicleCapacity(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory
            .forEach(Vehicle::class.java)
            .filter { vehicle -> vehicle.getTotalDemand() > vehicle.capacity }
            .penalizeLong(HardSoftLongScore.ONE_HARD) { vehicle ->
                (vehicle.getTotalDemand() - vehicle.capacity).toLong()
            }
            .asConstraint("Vehicle capacity")
    }

    fun timeWindowConstraint(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory
            .forEach(Visit::class.java)
            .filter { visit ->
                val vehicle = visit.vehicle
                val arrivalTime = calculateArrivalTime(vehicle, visit)
                arrivalTime > visit.maxEndTime
            }
            .penalizeLong(HardSoftLongScore.ONE_HARD) { visit ->
                val vehicle = visit.vehicle
                val arrivalTime = calculateArrivalTime(vehicle, visit)
                Duration.between(visit.maxEndTime, arrivalTime).inWholeSeconds
            }
            .asConstraint("Time window")
    }

    fun minimizeTravelTime(constraintFactory: ConstraintFactory): Constraint {
        return constraintFactory
            .forEach(Vehicle::class.java)
            .penalizeLong(HardSoftLongScore.ONE_SOFT) { vehicle ->
                vehicle.getTotalTravelTime().inWholeSeconds
            }
            .asConstraint("Minimize travel time")
    }

    private fun calculateArrivalTime(vehicle: Vehicle, visit: Visit): LocalDateTime {
        var currentTime = vehicle.startTime
        var currentLocation = vehicle.homeLocation
        for (v in vehicle.visits) {
            if (v == visit) break
            currentTime += Duration.seconds((currentLocation.calcEuclideanDistanceTo(v.location) * 3600 / 50).toLong())
            currentTime += v.serviceDuration
            currentLocation = v.location
        }
        currentTime += Duration.seconds((currentLocation.calcEuclideanDistanceTo(visit.location) * 3600 / 50).toLong())
        return currentTime
    }
}

In [ ]:
import ai.timefold.solver.core.api.solver.SolverFactory
import ai.timefold.solver.core.config.solver.SolverConfig
import kotlinx.datetime.LocalDateTime
import java.time.Duration
import com.fasterxml.jackson.module.kotlin.jacksonObjectMapper
import java.io.File

// Lire les données d'entrée
val mapper = jacksonObjectMapper()
val problem: VehicleRoutePlan = mapper.readValue(File("vehicle-routing-data.json"))

// Configurer et résoudre
val solverFactory = SolverFactory.create(
    SolverConfig().apply {
        withSolutionClass(VehicleRoutePlan::class.java)
        withEntityClasses(Vehicle::class.java)
        withConstraintProviderClass(VehicleRoutingConstraintProvider::class.java)
        withTerminationSpentLimit(Duration.ofSeconds(30))
    }
)

println("Résolution du problème...")
val solver = solverFactory.buildSolver()
val solution = solver.solve(problem)
println("Résolution terminée avec le score : ${solution.score}")

// Afficher la solution
solution.vehicles.forEach { vehicle ->
    println("${vehicle.name}: ${vehicle.visits.joinToString(" -> ") { it.name }}")
}